In [2]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")

import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np
from geojson.geometry import Point, Polygon, MultiLineString

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response


In [5]:
det_val = pd.read_csv('../cgarrido/berlin_data/traffic/det_val_hr_2022_01.csv', sep=';')

In [ ]:
det_val

In [20]:
mq_hr = pd.read_csv('../cgarrido/berlin_data/traffic/mq_hr_2022_01.csv', sep=';')

In [21]:
mq_hr

,mq_name,tag,stunde,qualitaet,q_kfz_mq_hr,v_kfz_mq_hr,q_pkw_mq_hr,v_pkw_mq_hr,q_lkw_mq_hr,v_lkw_mq_hr
0,TE004,04.01.2022,10,1.0,518,42,501,42,17,41
1,TE004,04.01.2022,11,1.0,610,40,596,40,14,42
2,TE004,04.01.2022,12,1.0,628,36,609,36,19,36
3,TE004,04.01.2022,13,1.0,624,41,598,41,26,38
4,TE004,04.01.2022,14,1.0,656,42,628,42,28,42
...,...,...,...,...,...,...,...,...,...,...
114673,TE559,31.01.2022,19,1.0,186,29,186,29,0,-1
114674,TE559,31.01.2022,20,1.0,128,28,128,28,0,-1
114675,TE559,31.01.2022,21,1.0,99,33,99,33,0,-1
114676,TE559,31.01.2022,22,1.0,101,34,100,34,1,26


In [22]:
traffic_stations = pd.read_excel('../cgarrido/berlin_data/traffic/traffic_station_locations.xlsx')

In [23]:
traffic_stations

,MQ_KURZNAME,DET_NAME_ALT,DET_NAME_NEU,DET_ID15,MQ_ID15,STRASSE,POSITION,POS_DETAIL,RICHTUNG,SPUR,annotation,LÄNGE (WGS84),BREITE (WGS84),INBETRIEBNAHME,ABBAUDATUM,DEINSTALLIERT,KOMMENTAR
0,TE001,TEU00002_Det0,TE001_Det_HF1,100101010000167,100201010000077,A115,AS Spanische Allee – Brücke,AK Zehlendorf,Südwest,HF_R,Hauptfahrbahn rechte Spur,13.192578,52.433868,2003-02-18,NaT,NaN,NaN
1,TE001,TEU00002_Det1,TE001_Det_HF2,100101010000268,100201010000077,A115,AS Spanische Allee – Brücke,AK Zehlendorf,Südwest,HF_2vR,"Hauptfahrbahn, 2. Spur von rechts",13.192578,52.433868,2003-02-18,NaT,NaN,NaN
2,TE002,TEU00002_Det2,TE002_Det_HF1,100101010000369,100201010000178,A115,AS Spanische Allee – Brücke,AD Funkturm,Nordost,HF_R,Hauptfahrbahn rechte Spur,13.192747,52.433813,2003-02-18,NaT,NaN,NaN
3,TE002,TEU00002_Det3,TE002_Det_HF2,100101010000470,100201010000178,A115,AS Spanische Allee – Brücke,AD Funkturm,Nordost,HF_2vR,"Hauptfahrbahn, 2. Spur von rechts",13.192747,52.433813,2003-02-18,NaT,NaN,NaN
4,TE004,TEU00004_Det0,TE004_Det_HF1,100101010000874,100201010000380,Clayallee,zwischen Scharfestraße und Propst-Süssmilch-We...,Potsdamer Chaussee,Süd,HF_R,Hauptfahrbahn rechte Spur,13.261301,52.436642,2003-02-18,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,TE583,teuscalaS00000DD00384D1,TE583_Det_HF2,100101010097975,100201010055348,Tempelhofer Damm,zwischen Burgemeisterstraße und Friedrich-Wilh...,Nord,Nord,HF_2vR,"Hauptfahrbahn, 2. Spur von rechts",13.384196,52.457440,2018-03-01,NaT,X,NaN
578,TE592,teuscalaS00000DD00391D0,TE592_Det_HF1,100101010099692,100201010056257,Kaiser-Friedrich-Straße,zwischen Schillerstraße und Pestalozzistraße,Süd,Süd,HF_R,Hauptfahrbahn rechte Spur,13.301719,52.509232,2018-04-27,NaT,X,NaN
579,TE592,teuscalaS00000DD00391D1,TE592_Det_HF2,100101010099793,100201010056257,Kaiser-Friedrich-Straße,zwischen Schillerstraße und Pestalozzistraße,Süd,Süd,HF_2vR,"Hauptfahrbahn, 2. Spur von rechts",13.301719,52.509232,2018-04-27,NaT,X,NaN
580,TE593,teuscalaS00000DD00392D0,TE593_Det_HF1,100101010099894,100201010056358,Kaiser-Friedrich-Straße,zwischen Pestalozzistraße und Schillerstraße,Nord,Nord,HF_R,Hauptfahrbahn rechte Spur,13.302183,52.508531,2018-04-27,NaT,X,NaN


In [ ]:
traffic_stations.columns

In [8]:
traffic_stn_thin = traffic_stations[['MQ_KURZNAME','LÄNGE (WGS84)', 'BREITE (WGS84)']].drop_duplicates()

In [9]:
traffic_stn_thin = traffic_stn_thin.rename(columns={"MQ_KURZNAME": "mq_name", "LÄNGE (WGS84)": "lat", "BREITE (WGS84)": "lon"})

In [10]:
mq_hr = mq_hr.merge(traffic_stn_thin, on = ['mq_name'])

In [11]:
mq_hr['stunde'].unique()

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,  0,  4,  5,
        6,  7,  8,  9,  1,  2,  3])

In [15]:
#qualitaet 	q_kfz_mq_hr 	v_kfz_mq_hr 	q_pkw_mq_hr 	v_pkw_mq_hr 	q_lkw_mq_hr 	v_lkw_mq_hr
num_rows_batch_publish=5000
entity_id_prefix =  "urn:ngsi-ld:Traffic:Berlin"
fac = True
for mq in list(mq_hr['mq_name'].unique()):
    entity_obj = Entity('TrafficObserved', entity_id_prefix + ':' + mq) 
    counter = 0
    station_tmp_gdf = mq_hr[mq_hr['mq_name'] == mq]
    for indx, itm in station_tmp_gdf.iterrows():
        obsdat = datetime.strptime(str(str(itm.tag) + ' ' + str(itm.stunde)), '%d.%m.%Y %H')
        entity_obj.add_prop("qualitaet",   str(itm['qualitaet'])   , observedAt= obsdat)
        entity_obj.add_prop("q_kfz_mq_hr", str(itm['q_kfz_mq_hr']) , observedAt= obsdat)
        entity_obj.add_prop("v_kfz_mq_hr", str(itm['v_kfz_mq_hr']) , observedAt= obsdat)
        entity_obj.add_prop("q_pkw_mq_hr", str(itm['q_pkw_mq_hr']) , observedAt= obsdat)
        entity_obj.add_prop("v_pkw_mq_hr", str(itm['v_pkw_mq_hr']) , observedAt= obsdat)
        entity_obj.add_prop("q_lkw_mq_hr", str(itm['q_lkw_mq_hr']) , observedAt= obsdat)
        entity_obj.add_prop("v_lkw_mq_hr", str(itm['v_lkw_mq_hr']) , observedAt= obsdat)
        entity_obj.add_geoprop("location", Point((itm.lat, itm.lon)), observedAt= obsdat)
        
        if fac == True :
            print(entity_obj)
            fac = False
            
        if (counter %num_rows_batch_publish==0 or counter == len(station_tmp_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
            publish(entity_obj.to_dict()) # publish the entity
            print('Published one batch')
            entity_obj = Entity('TrafficObserved', entity_id_prefix + ':' + mq)

        counter = counter + 1
print('published the data')    


{'id': 'urn:ngsi-ld:Traffic:Berlin:TE004', 'type': 'TrafficObserved', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'qualitaet': {'type': 'Property', 'value': '1.0', 'observedAt': '2022-01-04T10:00:00Z'}, 'q_kfz_mq_hr': {'type': 'Property', 'value': '518', 'observedAt': '2022-01-04T10:00:00Z'}, 'v_kfz_mq_hr': {'type': 'Property', 'value': '42', 'observedAt': '2022-01-04T10:00:00Z'}, 'q_pkw_mq_hr': {'type': 'Property', 'value': '501', 'observedAt': '2022-01-04T10:00:00Z'}, 'v_pkw_mq_hr': {'type': 'Property', 'value': '42', 'observedAt': '2022-01-04T10:00:00Z'}, 'q_lkw_mq_hr': {'type': 'Property', 'value': '17', 'observedAt': '2022-01-04T10:00:00Z'}, 'v_lkw_mq_hr': {'type': 'Property', 'value': '41', 'observedAt': '2022-01-04T10:00:00Z'}, 'location': {'type': 'GeoProperty', 'value': {"coordinates": [13.261301, 52.436642], "type": "Point"}, 'observedAt': '2022-01-04T10:00:00Z'}}
Published one batch
Published one batch
Published one batch
Published one batch


KeyboardInterrupt: 

In [19]:
mq_hr.to_pickle('../cgarrido/berlin_data/traffic/mq_hr_with_location.pkl')